In [1]:
import pandas as pd
import string

from sklearn.preprocessing import LabelEncoder

In [41]:
# Read in Data
data = pd.read_csv('holdout_set.csv', encoding='cp1252')
data.head()

,Engagements,Followers at Posting,Created,Type,Description
0,443543,36984682,2019-05-22 00:31:55 EDT,Photo,The @raptors even up the Eastern Conference Fi...
1,443543,36984682,2019-05-21 23:20:41 EDT,Photo,The @raptors even the Eastern Conference Final...
2,603380,36984682,2019-05-21 21:18:51 EDT,Video,@sergeibaka keeps it alive for the @raptors! (...
3,675091,36955156,2019-05-21 18:20:09 EDT,Video,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...
4,675091,36955156,2019-05-21 15:04:31 EDT,Video,?? the @raptors & @bucks in transition! #NBABr...


In [42]:
# Split data to Year/Month/Day Hour/Minute/Seconds columns
data['Year'] = data['Created'].apply(lambda date: date.split('-')[0].strip())
data['Month'] = data['Created'].apply(lambda date: date.split('-')[1].strip())
data['Day'] = data['Created'].apply(lambda date: date.split('-')[2][:2].strip())

data['Hour'] = data['Created'].apply(lambda date: date.split(' ')[1].split(':')[0].strip())
data['Minute'] = data['Created'].apply(lambda date: date.split(' ')[1].split(':')[1].strip())
data['Second'] = data['Created'].apply(lambda date: date.split(' ')[1].split(':')[2].strip())

del data['Created']
data.head(10)

,Engagements,Followers at Posting,Type,Description,Year,Month,Day,Hour,Minute,Second
0,443543,36984682,Photo,The @raptors even up the Eastern Conference Fi...,2019,05,22,00,31,55
1,443543,36984682,Photo,The @raptors even the Eastern Conference Final...,2019,05,21,23,20,41
2,603380,36984682,Video,@sergeibaka keeps it alive for the @raptors! (...,2019,05,21,21,18,51
3,675091,36955156,Video,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,2019,05,21,18,20,09
4,675091,36955156,Video,?? the @raptors & @bucks in transition! #NBABr...,2019,05,21,15,04,31
5,443543,36955156,Photo,The @warriors advance to the 2019 #NBAFinals p...,2019,05,21,02,09,47
6,675091,36955156,Video,Big bro wins this one! #PhantomCam,2019,05,21,01,30,13
7,675091,36955156,Video,@stephencurry30 pulls up from deep to end the ...,2019,05,20,22,25,20
8,675091,36955156,Video,The @warriors move the ball to find Andrew Bog...,2019,05,20,22,09,36
9,675091,36928789,Video,?? @money23green mic’d up in Game 3 of the WCF...,2019,05,20,15,26,07


In [45]:
# Label Encode Type Column
label_encoder = LabelEncoder()
data['Type'] = label_encoder.fit_transform(data['Type'])

data.head()

,Engagements,Followers at Posting,Type,Description,Year,Month,Day,Hour,Minute,Second
0,443543,36984682,1,The @raptors even up the Eastern Conference Fi...,2019,05,22,00,31,55
1,443543,36984682,1,The @raptors even the Eastern Conference Final...,2019,05,21,23,20,41
2,603380,36984682,2,@sergeibaka keeps it alive for the @raptors! (...,2019,05,21,21,18,51
3,675091,36955156,2,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,2019,05,21,18,20,09
4,675091,36955156,2,?? the @raptors & @bucks in transition! #NBABr...,2019,05,21,15,04,31


In [49]:
# Strip whitespace from Description before any other processing
data['Description'] = data['Description'].apply(lambda desc: desc.strip())
data.head(7)

,Engagements,Followers at Posting,Type,Description,Year,Month,Day,Hour,Minute,Second
0,443543,36984682,1,The @raptors even up the Eastern Conference Fi...,2019,05,22,00,31,55
1,443543,36984682,1,The @raptors even the Eastern Conference Final...,2019,05,21,23,20,41
2,603380,36984682,2,@sergeibaka keeps it alive for the @raptors! (...,2019,05,21,21,18,51
3,675091,36955156,2,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,2019,05,21,18,20,09
4,675091,36955156,2,?? the @raptors & @bucks in transition! #NBABr...,2019,05,21,15,04,31
5,443543,36955156,1,The @warriors advance to the 2019 #NBAFinals p...,2019,05,21,02,09,47
6,675091,36955156,2,Big bro wins this one! #PhantomCam,2019,05,21,01,30,13


In [56]:
def remove_punctuation(text):
    if type(text) != str:
        return ''
    return text.translate(str.maketrans('', '', string.punctuation))

data['Description_Clean'] = data['Description'].apply(remove_punctuation)
data.head()

,Engagements,Followers at Posting,Type,Description,Year,Month,Day,Hour,Minute,Second,Description_Clean
0,443543,36984682,1,The @raptors even up the Eastern Conference Fi...,2019,05,22,00,31,55,The raptors even up the Eastern Conference Fin...
1,443543,36984682,1,The @raptors even the Eastern Conference Final...,2019,05,21,23,20,41,The raptors even the Eastern Conference Finals...
2,603380,36984682,2,@sergeibaka keeps it alive for the @raptors! (...,2019,05,21,21,18,51,sergeibaka keeps it alive for the raptors nba...
3,675091,36955156,2,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,2019,05,21,18,20,09,The SUPERHUMAN HANDS of Giannis Kawhi \n\n b...
4,675091,36955156,2,?? the @raptors & @bucks in transition! #NBABr...,2019,05,21,15,04,31,the raptors bucks in transition NBABreakdown...


In [59]:
data['Description_Clean'][0]

'The raptors even up the Eastern Conference Finals 22'